# Challenge 1 
### Team Name: 4-mille-feuille
#### Students: 
    - Thomas A. Fernandez Ramirez (Student ID: 473540)
    - Sander Lindberg (Student ID: 493389)

## Import

In [43]:
import pandas as pd
import numpy as np

## Loading and creating the different sets for training
The dataset has from the Long_submission removed nan values.

In [44]:
X_train_pre = pd.read_csv('train_preprocessed_no_nan.csv')
X_test_pre = pd.read_csv('test_preprocessed_no_nan.csv')

X_train = X_train_pre.drop(['target', 'id'], axis=1)
y = X_train_pre['target']

X_test = X_test_pre.copy()

In [45]:
X_train.head()

,f0,f1,f2,f3,f4,f5,f7,f8,f10,f12,...,f20,f21,f22,f23,f24,f25,f27,f28,f9_1,f9_2
0,8,0,-1.435379,0.862103,6.8928,1,2,4,1,0,...,0.888292,3,0,1,0,51,12,1,15,18
1,13,-1,-1.219974,-0.915560,6.8098,0,1,6,10,5,...,-0.148665,3,1,5,0,10,10,1,3,19
2,15,4,-0.326541,0.238072,6.7761,0,2,5,0,2,...,0.909928,3,0,2,1,16,2,0,46,41
3,10,3,0.341725,1.268089,6.9302,0,2,5,1,0,...,1.189065,2,0,3,0,63,8,1,1,47
4,7,0,0.089125,1.542401,7.0238,0,1,5,0,3,...,0.658525,2,1,0,1,26,7,1,35,1


In [46]:
X_test.head()

,f0,f1,f2,f3,f4,f5,f7,f8,f10,f12,...,f20,f21,f22,f23,f24,f25,f27,f28,f9_1,f9_2
0,23,-1,0.534264,1.701243,7.0036,0,2,5,1,2,...,-0.505200,3,1,4,0,104,14,1,18,49
1,2,1,-0.869325,-0.472949,6.9520,0,2,4,1,3,...,1.617640,1,0,4,0,20,7,0,1,11
2,16,1,1.176812,-0.245159,6.8457,0,2,1,0,0,...,-1.131335,2,0,5,0,28,10,0,19,25
3,7,-1,1.720175,-0.223120,6.9464,1,2,4,1,0,...,1.303898,2,1,4,1,64,0,1,0,6
4,2,-1,0.048927,0.052296,7.2400,0,1,1,0,1,...,0.403987,1,1,0,0,76,3,1,17,49


In [47]:
y.head()

0    0
1    1
2    0
3    0
4    0
Name: target, dtype: int64

In [48]:
X_train.drop(columns=['f9_2', 'f2', 'f4', 'f13', 'f22'], inplace=True)

In [49]:
X_test.drop(columns=['f9_2', 'f2', 'f4', 'f13', 'f22'], inplace=True)

## Functions for simplification of code

In [50]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

def split(X : pd.DataFrame, y : pd.DataFrame, size=.2):
    return train_test_split(X, y, test_size=size, stratify=y)

def fit(data : pd.DataFrame, labels : pd.DataFrame, classifier : any, eval_pool = None) -> None:
    if eval_pool is not None:
        classifier.fit(data, labels, eval_set=eval_pool)
    else:
        classifier.fit(data, labels)

def predict_proba(classifier : any, test : pd.DataFrame) -> np.array:
    return classifier.predict_proba(test)

def predict(classifier : any, test : pd.DataFrame) -> np.array:
    return classifier.predict(test)

def print_accuracy(pred : np.array, test : pd.DataFrame, name : str):
    print(name + ' Model AOC score: {0:0.4f}'.format(roc_auc_score(test, pred[:, 1])))
    
def real_predict(clf, name, submission=True):
    prediction = predict_proba(clf, X_test_split)

    print_accuracy(prediction, y_test_split, name)

    if submission:
        real_pred = predict_proba(clf, X_test)

        f = open("./pred_Newpara.csv", "w")
        f.write("id,target\n")
        id_nr = 50000
        for v in real_pred[:, 1]:
            f.write(f"{id_nr},{v}\n")
            id_nr += 1
        f.close()

In [51]:
X_train_split, X_test_split, y_train_split, y_test_split = split(X_train, y, size=.1)

In [52]:
from catboost import CatBoostClassifier, Pool

n = len(X_train)

class_0 = len(X_train_pre[X_train_pre['target'] == 0])
class_1 = len(X_train_pre[X_train_pre['target'] == 1])

eval_pool = Pool(X_test_split, y_test_split)

border_count=166
depth_found=6.0 
l2_leaf_reg_found=5.5
lr=0.0967
nb_trees=1351

clf = CatBoostClassifier(depth=depth_found,use_best_model=True,eval_metric='AUC',od_type="Iter", border_count=border_count, learning_rate=lr, iterations=2000, l2_leaf_reg = l2_leaf_reg_found ,leaf_estimation_iterations=10, loss_function= 'Logloss')

fit(X_train, y, clf, eval_pool)

0:	test: 0.6710644	best: 0.6710644 (0)	total: 32.4ms	remaining: 1m 4s
1:	test: 0.7014699	best: 0.7014699 (1)	total: 69ms	remaining: 1m 8s
2:	test: 0.7106134	best: 0.7106134 (2)	total: 84.8ms	remaining: 56.5s
3:	test: 0.7210052	best: 0.7210052 (3)	total: 102ms	remaining: 51s
4:	test: 0.7207454	best: 0.7210052 (3)	total: 140ms	remaining: 55.9s
5:	test: 0.7305899	best: 0.7305899 (5)	total: 153ms	remaining: 50.9s
6:	test: 0.7322413	best: 0.7322413 (6)	total: 178ms	remaining: 50.6s
7:	test: 0.7309772	best: 0.7322413 (6)	total: 197ms	remaining: 49s
8:	test: 0.7371581	best: 0.7371581 (8)	total: 214ms	remaining: 47.4s
9:	test: 0.7405029	best: 0.7405029 (9)	total: 231ms	remaining: 46s
10:	test: 0.7420345	best: 0.7420345 (10)	total: 249ms	remaining: 45s
11:	test: 0.7431027	best: 0.7431027 (11)	total: 267ms	remaining: 44.2s
12:	test: 0.7433458	best: 0.7433458 (12)	total: 287ms	remaining: 43.8s
13:	test: 0.7442872	best: 0.7442872 (13)	total: 297ms	remaining: 42.2s
14:	test: 0.7457386	best: 0.74573

119:	test: 0.7822513	best: 0.7822513 (119)	total: 2.1s	remaining: 32.9s
120:	test: 0.7824454	best: 0.7824454 (120)	total: 2.12s	remaining: 32.9s
121:	test: 0.7825884	best: 0.7825884 (121)	total: 2.14s	remaining: 32.9s
122:	test: 0.7827311	best: 0.7827311 (122)	total: 2.15s	remaining: 32.8s
123:	test: 0.7830100	best: 0.7830100 (123)	total: 2.17s	remaining: 32.8s
124:	test: 0.7833649	best: 0.7833649 (124)	total: 2.18s	remaining: 32.8s
125:	test: 0.7835171	best: 0.7835171 (125)	total: 2.2s	remaining: 32.8s
126:	test: 0.7836431	best: 0.7836431 (126)	total: 2.22s	remaining: 32.8s
127:	test: 0.7839262	best: 0.7839262 (127)	total: 2.24s	remaining: 32.8s
128:	test: 0.7842601	best: 0.7842601 (128)	total: 2.26s	remaining: 32.8s
129:	test: 0.7844798	best: 0.7844798 (129)	total: 2.27s	remaining: 32.7s
130:	test: 0.7847015	best: 0.7847015 (130)	total: 2.29s	remaining: 32.7s
131:	test: 0.7848771	best: 0.7848771 (131)	total: 2.31s	remaining: 32.7s
132:	test: 0.7854091	best: 0.7854091 (132)	total: 2.3

232:	test: 0.8078164	best: 0.8078164 (232)	total: 4.38s	remaining: 33.2s
233:	test: 0.8080569	best: 0.8080569 (233)	total: 4.39s	remaining: 33.2s
234:	test: 0.8081992	best: 0.8081992 (234)	total: 4.42s	remaining: 33.2s
235:	test: 0.8084318	best: 0.8084318 (235)	total: 4.44s	remaining: 33.2s
236:	test: 0.8085548	best: 0.8085548 (236)	total: 4.46s	remaining: 33.2s
237:	test: 0.8088259	best: 0.8088259 (237)	total: 4.49s	remaining: 33.2s
238:	test: 0.8088721	best: 0.8088721 (238)	total: 4.51s	remaining: 33.2s
239:	test: 0.8089345	best: 0.8089345 (239)	total: 4.53s	remaining: 33.2s
240:	test: 0.8090075	best: 0.8090075 (240)	total: 4.55s	remaining: 33.2s
241:	test: 0.8093727	best: 0.8093727 (241)	total: 4.57s	remaining: 33.2s
242:	test: 0.8095536	best: 0.8095536 (242)	total: 4.6s	remaining: 33.3s
243:	test: 0.8099011	best: 0.8099011 (243)	total: 4.62s	remaining: 33.2s
244:	test: 0.8102231	best: 0.8102231 (244)	total: 4.63s	remaining: 33.2s
245:	test: 0.8104618	best: 0.8104618 (245)	total: 4.

349:	test: 0.8289662	best: 0.8289662 (349)	total: 7.12s	remaining: 33.6s
350:	test: 0.8292152	best: 0.8292152 (350)	total: 7.15s	remaining: 33.6s
351:	test: 0.8294337	best: 0.8294337 (351)	total: 7.16s	remaining: 33.5s
352:	test: 0.8295359	best: 0.8295359 (352)	total: 7.18s	remaining: 33.5s
353:	test: 0.8297560	best: 0.8297560 (353)	total: 7.19s	remaining: 33.4s
354:	test: 0.8299720	best: 0.8299720 (354)	total: 7.21s	remaining: 33.4s
355:	test: 0.8301508	best: 0.8301508 (355)	total: 7.23s	remaining: 33.4s
356:	test: 0.8303225	best: 0.8303225 (356)	total: 7.24s	remaining: 33.3s
357:	test: 0.8304358	best: 0.8304358 (357)	total: 7.26s	remaining: 33.3s
358:	test: 0.8305370	best: 0.8305370 (358)	total: 7.3s	remaining: 33.4s
359:	test: 0.8307087	best: 0.8307087 (359)	total: 7.35s	remaining: 33.5s
360:	test: 0.8307773	best: 0.8307773 (360)	total: 7.37s	remaining: 33.5s
361:	test: 0.8309634	best: 0.8309634 (361)	total: 7.39s	remaining: 33.4s
362:	test: 0.8310870	best: 0.8310870 (362)	total: 7.

471:	test: 0.8490653	best: 0.8490653 (471)	total: 9.71s	remaining: 31.4s
472:	test: 0.8492236	best: 0.8492236 (472)	total: 9.72s	remaining: 31.4s
473:	test: 0.8492883	best: 0.8492883 (473)	total: 9.75s	remaining: 31.4s
474:	test: 0.8495109	best: 0.8495109 (474)	total: 9.76s	remaining: 31.3s
475:	test: 0.8496784	best: 0.8496784 (475)	total: 9.79s	remaining: 31.3s
476:	test: 0.8498377	best: 0.8498377 (476)	total: 9.8s	remaining: 31.3s
477:	test: 0.8499168	best: 0.8499168 (477)	total: 9.82s	remaining: 31.3s
478:	test: 0.8500099	best: 0.8500099 (478)	total: 9.84s	remaining: 31.2s
479:	test: 0.8501492	best: 0.8501492 (479)	total: 9.85s	remaining: 31.2s
480:	test: 0.8502136	best: 0.8502136 (480)	total: 9.87s	remaining: 31.2s
481:	test: 0.8502954	best: 0.8502954 (481)	total: 9.89s	remaining: 31.2s
482:	test: 0.8504642	best: 0.8504642 (482)	total: 9.93s	remaining: 31.2s
483:	test: 0.8504623	best: 0.8504642 (482)	total: 9.94s	remaining: 31.1s
484:	test: 0.8506014	best: 0.8506014 (484)	total: 9.

...

1522:	test: 0.9434018	best: 0.9434018 (1522)	total: 29.7s	remaining: 9.29s
1523:	test: 0.9435745	best: 0.9435745 (1523)	total: 29.7s	remaining: 9.27s
1524:	test: 0.9436747	best: 0.9436747 (1524)	total: 29.7s	remaining: 9.25s
1525:	test: 0.9436868	best: 0.9436868 (1525)	total: 29.7s	remaining: 9.23s
1526:	test: 0.9437168	best: 0.9437168 (1526)	total: 29.7s	remaining: 9.21s
1527:	test: 0.9437843	best: 0.9437843 (1527)	total: 29.8s	remaining: 9.19s
1528:	test: 0.9438409	best: 0.9438409 (1528)	total: 29.8s	remaining: 9.17s
1529:	test: 0.9439381	best: 0.9439381 (1529)	total: 29.8s	remaining: 9.15s
1530:	test: 0.9439923	best: 0.9439923 (1530)	total: 29.8s	remaining: 9.13s
1531:	test: 0.9439991	best: 0.9439991 (1531)	total: 29.8s	remaining: 9.11s
1532:	test: 0.9441035	best: 0.9441035 (1532)	total: 29.8s	remaining: 9.09s
1533:	test: 0.9441432	best: 0.9441432 (1533)	total: 29.9s	remaining: 9.07s
1534:	test: 0.9442241	best: 0.9442241 (1534)	total: 29.9s	remaining: 9.05s
1535:	test: 0.9443548	bes

1636:	test: 0.9498360	best: 0.9498360 (1636)	total: 31.8s	remaining: 7.04s
1637:	test: 0.9498528	best: 0.9498528 (1637)	total: 31.8s	remaining: 7.03s
1638:	test: 0.9498925	best: 0.9498925 (1638)	total: 31.8s	remaining: 7s
1639:	test: 0.9498865	best: 0.9498925 (1638)	total: 31.8s	remaining: 6.98s
1640:	test: 0.9499225	best: 0.9499225 (1640)	total: 31.8s	remaining: 6.96s
1641:	test: 0.9499467	best: 0.9499467 (1641)	total: 31.8s	remaining: 6.94s
1642:	test: 0.9500660	best: 0.9500660 (1642)	total: 31.9s	remaining: 6.92s
1643:	test: 0.9501276	best: 0.9501276 (1643)	total: 31.9s	remaining: 6.91s
1644:	test: 0.9501680	best: 0.9501680 (1644)	total: 31.9s	remaining: 6.88s
1645:	test: 0.9502645	best: 0.9502645 (1645)	total: 31.9s	remaining: 6.87s
1646:	test: 0.9502771	best: 0.9502771 (1646)	total: 31.9s	remaining: 6.85s
1647:	test: 0.9503447	best: 0.9503447 (1647)	total: 32s	remaining: 6.83s
1648:	test: 0.9503907	best: 0.9503907 (1648)	total: 32s	remaining: 6.8s
1649:	test: 0.9503941	best: 0.950

1749:	test: 0.9549189	best: 0.9549189 (1749)	total: 33.6s	remaining: 4.8s
1750:	test: 0.9548669	best: 0.9549189 (1749)	total: 33.6s	remaining: 4.78s
1751:	test: 0.9548879	best: 0.9549189 (1749)	total: 33.6s	remaining: 4.76s
1752:	test: 0.9549873	best: 0.9549873 (1752)	total: 33.6s	remaining: 4.74s
1753:	test: 0.9550323	best: 0.9550323 (1753)	total: 33.7s	remaining: 4.72s
1754:	test: 0.9550725	best: 0.9550725 (1754)	total: 33.7s	remaining: 4.7s
1755:	test: 0.9550961	best: 0.9550961 (1755)	total: 33.7s	remaining: 4.68s
1756:	test: 0.9551224	best: 0.9551224 (1756)	total: 33.7s	remaining: 4.66s
1757:	test: 0.9551403	best: 0.9551403 (1757)	total: 33.7s	remaining: 4.64s
1758:	test: 0.9551742	best: 0.9551742 (1758)	total: 33.7s	remaining: 4.62s
1759:	test: 0.9552081	best: 0.9552081 (1759)	total: 33.7s	remaining: 4.6s
1760:	test: 0.9552894	best: 0.9552894 (1760)	total: 33.7s	remaining: 4.58s
1761:	test: 0.9553401	best: 0.9553401 (1761)	total: 33.8s	remaining: 4.56s
1762:	test: 0.9554505	best: 

1866:	test: 0.9601738	best: 0.9601738 (1866)	total: 35.7s	remaining: 2.54s
1867:	test: 0.9601554	best: 0.9601738 (1866)	total: 35.7s	remaining: 2.52s
1868:	test: 0.9601383	best: 0.9601738 (1866)	total: 35.7s	remaining: 2.5s
1869:	test: 0.9601962	best: 0.9601962 (1869)	total: 35.7s	remaining: 2.48s
1870:	test: 0.9602316	best: 0.9602316 (1870)	total: 35.7s	remaining: 2.46s
1871:	test: 0.9602196	best: 0.9602316 (1870)	total: 35.7s	remaining: 2.44s
1872:	test: 0.9602855	best: 0.9602855 (1872)	total: 35.7s	remaining: 2.42s
1873:	test: 0.9603231	best: 0.9603231 (1873)	total: 35.8s	remaining: 2.4s
1874:	test: 0.9603510	best: 0.9603510 (1874)	total: 35.8s	remaining: 2.38s
1875:	test: 0.9604149	best: 0.9604149 (1875)	total: 35.8s	remaining: 2.37s
1876:	test: 0.9605316	best: 0.9605316 (1876)	total: 35.8s	remaining: 2.35s
1877:	test: 0.9605808	best: 0.9605808 (1877)	total: 35.8s	remaining: 2.33s
1878:	test: 0.9606791	best: 0.9606791 (1878)	total: 35.8s	remaining: 2.31s
1879:	test: 0.9606775	best:

1977:	test: 0.9644176	best: 0.9644176 (1977)	total: 37.5s	remaining: 417ms
1978:	test: 0.9644279	best: 0.9644279 (1978)	total: 37.5s	remaining: 398ms
1979:	test: 0.9644991	best: 0.9644991 (1979)	total: 37.6s	remaining: 379ms
1980:	test: 0.9645186	best: 0.9645186 (1980)	total: 37.6s	remaining: 360ms
1981:	test: 0.9645094	best: 0.9645186 (1980)	total: 37.6s	remaining: 341ms
1982:	test: 0.9645370	best: 0.9645370 (1982)	total: 37.6s	remaining: 322ms
1983:	test: 0.9645661	best: 0.9645661 (1983)	total: 37.6s	remaining: 303ms
1984:	test: 0.9645882	best: 0.9645882 (1984)	total: 37.6s	remaining: 284ms
1985:	test: 0.9645798	best: 0.9645882 (1984)	total: 37.6s	remaining: 265ms
1986:	test: 0.9646742	best: 0.9646742 (1986)	total: 37.6s	remaining: 246ms
1987:	test: 0.9646660	best: 0.9646742 (1986)	total: 37.7s	remaining: 227ms
1988:	test: 0.9646713	best: 0.9646742 (1986)	total: 37.7s	remaining: 208ms
1989:	test: 0.9647241	best: 0.9647241 (1989)	total: 37.7s	remaining: 189ms
1990:	test: 0.9647212	bes

## Test and create submission

In [54]:
real_predict(clf, 'Catboost')

Catboost Model AOC score: 0.9651
